In [ ]:
import pandas as pd
import numpy as np

ncnty = 19
nyear =1
n1 = 6860
nn = 7220

# Load data
trade1 = pd.read_csv('c:\\research\\eaton\\tgt\\trade1', header=None)

# Select 1990 data
trade1 = trade1.iloc[6859:7219, :]

# Extract variables
impcnty = trade1.iloc[:, 0]
expcnty = trade1.iloc[:, 1]
depvar = trade1.iloc[:, 3]
depvarn = trade1.iloc[:, 4]
dist1 = trade1.iloc[:, 5]
dist2 = trade1.iloc[:, 6]
dist3 = trade1.iloc[:, 7]
dist4 = trade1.iloc[:, 8]
dist5 = trade1.iloc[:, 9]
dist6 = trade1.iloc[:, 10]
border = trade1.iloc[:, 11]
lrrnd = trade1.iloc[:, 12]
lrhk = trade1.iloc[:, 13]
lrwork = trade1.iloc[:, 15]
lrwage = trade1.iloc[:, 16]

mybeta = .21221
diff = -(depvarn-depvar)*mybeta/(1-mybeta)
depvarp = depvar + diff

# Create common language variable
english = np.array([1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1])
french = np.array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
german = np.array([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
talk = english @ english + french @ french + german @ german
talk = np.ones((20, 1)) * talk

# Create country trade blocks
countries = ['AL', 'AU', 'BE', 'CA', 'DE', 'FI', 'FR', 'GE', 'GR', 'IT', 'JA', 'NE', 'NZ', 'NO', 'PO', 'SP', 'SW', 'UK', 'US']

eu71 = np.array([0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0])
eu73 = np.array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])
eu81 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
eu86 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0])

eu7172 = np.ones((2,1)) * np.sum(eu71 * eu71)
eu7380 = np.ones((8,1)) * np.sum((eu71 + eu73) * (eu71 + eu73))
eu8185 = np.ones((5,1)) * np.sum((eu71 + eu73 + eu81) * (eu71 + eu73 + eu81))
eu8690 = np.ones((5,1)) * np.sum((eu71 + eu73 + eu81 + eu86) * (eu71 + eu73 + eu81 + eu86))

eu = np.concatenate((eu7172, eu7380, eu8185, eu8690))

ef71 = np.array([0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0])
ef73 = np.array([0, 0, 0, 0,-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,-1, 0])
ef86 = np.array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,-1,-1, 0, 0, 0])

ef7172 = np.ones((2,1)) * np.sum(ef71 * ef71)
ef7385 = np.ones((13,1)) * np.sum((ef71 + ef73) * (ef71 + ef73))
ef8690 = np.ones((5,1)) * np.sum((ef71 + ef73 + ef86) * (ef71 + ef73 + ef86))

ef = np.concatenate((ef7172, ef7385, ef8690))

# Create the source and destination dummy variables
onecnty = np.ones((ncnty, 1))
desdum = np.ones((nyear, 1)) @ (np.eye(ncnty) @ onecnty)
srcdum = np.ones((nyear, 1)) @ (onecnty.T @ np.eye(ncnty))
reldum = srcdum - desdum

desdumr = np.hstack((desdum[:, :ncnty-1], desdum[:, ncnty]))
reldumr = np.hstack((reldum[:, :ncnty-1], reldum[:, ncnty]))

# Function to calculate country pair time averages
def order2(datf):
    adatf = np.ones((ncnty * (ncnty - 1), datf.shape[1]))
    for i in range(datf.shape[1]):
        var = datf[:, i]
        varmat = var.reshape(nyear, ncnty * (ncnty - 1))
        adatf[:, i] = varmat.mean(axis=0)
    return adatf

In [ ]:
import statsmodels.api as sm

def gls(y, x, icov):
    # Perform GLS regression
    gls_model = sm.GLS(y, x, sigma=icov)
    results = gls_model.fit()

    # Calculate residuals
    eps = y - np.dot(x, results.params)

    print("Raw SS in this regression ", np.dot(eps.T, eps))
    print("Adjusted SS ", np.dot(np.dot(eps.T, icov), eps))

    # Calculate source and destination effects for USA
    print("source effect: usa=", -np.ones((1, 18)) @ results.params[10:28])
    print("source effect: stdb(usa)=", np.sqrt(np.ones((1, 18)) @ results.normalized_cov_params[10:28, 10:28] @ np.ones((18, 1))))
    print("destination effect: usa=", -np.ones((1, 18)) @ results.params[28:46])
    print("destination effect: stdb(usa)=", np.sqrt(np.ones((1, 18)) @ results.normalized_cov_params[28:46, 28:46] @ np.ones((18, 1))))

    stdb = np.sqrt(np.diag(results.normalized_cov_params))

    return results.params, stdb, eps